<a href="https://www.kaggle.com/code/ahmedanwar89/supermarket-sales-eda?scriptVersionId=151820510" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# Import Libraries

In [1]:
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Import Dataset

In [2]:
df = pd.read_csv('/kaggle/input/supermarket-sales/supermarket_sales - Sheet1.csv')

In [3]:
df.sample(5)

,Invoice ID,Branch,City,Customer type,Gender,Product line,Unit price,Quantity,Tax 5%,Total,Date,Time,Payment,cogs,gross margin percentage,gross income,Rating
562,134-54-4720,B,Mandalay,Normal,Female,Electronic accessories,42.42,8,16.9680,356.3280,1/30/2019,13:58,Ewallet,339.36,4.761905,16.9680,5.7
536,339-12-4827,B,Mandalay,Member,Female,Fashion accessories,73.96,1,3.6980,77.6580,1/5/2019,11:32,Credit card,73.96,4.761905,3.6980,5.0
854,160-22-2687,A,Yangon,Member,Female,Health and beauty,95.95,5,23.9875,503.7375,1/23/2019,14:21,Ewallet,479.75,4.761905,23.9875,8.8
478,701-69-8742,B,Mandalay,Normal,Male,Sports and travel,34.37,10,17.1850,360.8850,3/16/2019,10:11,Ewallet,343.70,4.761905,17.1850,6.7
337,692-27-8933,B,Mandalay,Normal,Female,Sports and travel,57.95,6,17.3850,365.0850,2/24/2019,13:02,Cash,347.70,4.761905,17.3850,5.2


# Data Clean

In [4]:
# check column names to if they with the best practices.
# replace the middle white spaces with '_' & and make all names in lower case.
df.columns = df.columns.str.replace(' ', '_', regex=True).str.lower()

In [5]:
# check the changes
df.columns

Index(['invoice_id', 'branch', 'city', 'customer_type', 'gender',
       'product_line', 'unit_price', 'quantity', 'tax_5%', 'total', 'date',
       'time', 'payment', 'cogs', 'gross_margin_percentage', 'gross_income',
       'rating'],
      dtype='object')

In [6]:
# check data type
df.dtypes

invoice_id                  object
branch                      object
city                        object
customer_type               object
gender                      object
product_line                object
unit_price                 float64
quantity                     int64
tax_5%                     float64
total                      float64
date                        object
time                        object
payment                     object
cogs                       float64
gross_margin_percentage    float64
gross_income               float64
rating                     float64
dtype: object

In [7]:
# create new column named 'date_time' for analysis latter.
df['date_time'] = df['date'] + ' ' + df['time']

In [8]:
# check changes.
df['date_time']

0       1/5/2019 13:08
1       3/8/2019 10:29
2       3/3/2019 13:23
3      1/27/2019 20:33
4       2/8/2019 10:37
            ...       
995    1/29/2019 13:46
996     3/2/2019 17:16
997     2/9/2019 13:22
998    2/22/2019 15:33
999    2/18/2019 13:28
Name: date_time, Length: 1000, dtype: object

In [9]:
# change 'date_time' to datetime.
df['date_time'] = pd.to_datetime(df['date_time'], utc=True, errors='coerce')

In [10]:
# check changes.
df['date_time']

0     2019-01-05 13:08:00+00:00
1     2019-03-08 10:29:00+00:00
2     2019-03-03 13:23:00+00:00
3     2019-01-27 20:33:00+00:00
4     2019-02-08 10:37:00+00:00
                 ...           
995   2019-01-29 13:46:00+00:00
996   2019-03-02 17:16:00+00:00
997   2019-02-09 13:22:00+00:00
998   2019-02-22 15:33:00+00:00
999   2019-02-18 13:28:00+00:00
Name: date_time, Length: 1000, dtype: datetime64[ns, UTC]

In [11]:
# drop not needed columns
df.drop(columns=['date', 'time'], inplace=True)

In [12]:
# check null values.
df.isnull().any(), df.isnull().sum()

(invoice_id                 False
 branch                     False
 city                       False
 customer_type              False
 gender                     False
 product_line               False
 unit_price                 False
 quantity                   False
 tax_5%                     False
 total                      False
 payment                    False
 cogs                       False
 gross_margin_percentage    False
 gross_income               False
 rating                     False
 date_time                  False
 dtype: bool,
 invoice_id                 0
 branch                     0
 city                       0
 customer_type              0
 gender                     0
 product_line               0
 unit_price                 0
 quantity                   0
 tax_5%                     0
 total                      0
 payment                    0
 cogs                       0
 gross_margin_percentage    0
 gross_income               0
 rating               

In [13]:
# check duplicated values.
df.duplicated().any(), df.duplicated().sum()

(False, 0)

In [14]:
# check data validity for object columns.
# first in loop check if column is in object type and have unique values equal or less than 15
# then if condition is true print column name and its unique values.
for x in df.columns :
    if (df[x].dtypes == 'object') & (df[x].nunique() <= 15) :
        print(x)
        print(df[x].unique())

branch
['A' 'C' 'B']
city
['Yangon' 'Naypyitaw' 'Mandalay']
customer_type
['Member' 'Normal']
gender
['Female' 'Male']
product_line
['Health and beauty' 'Electronic accessories' 'Home and lifestyle'
 'Sports and travel' 'Food and beverages' 'Fashion accessories']
payment
['Ewallet' 'Cash' 'Credit card']


In [15]:
# check data validity for numerical columns.
# frist select numerical columns from df and exclude object and date columns by select_dtypes()
# then apply describe() & round() by 2.
df.select_dtypes(exclude=['object', 'datetime64[ns, UTC]']).describe().round(2)

,unit_price,quantity,tax_5%,total,cogs,gross_margin_percentage,gross_income,rating
count,1000.00,1000.00,1000.00,1000.00,1000.00,1000.00,1000.00,1000.00
mean,55.67,5.51,15.38,322.97,307.59,4.76,15.38,6.97
std,26.49,2.92,11.71,245.89,234.18,0.00,11.71,1.72
min,10.08,1.00,0.51,10.68,10.17,4.76,0.51,4.00
25%,32.88,3.00,5.92,124.42,118.50,4.76,5.92,5.50
50%,55.23,5.00,12.09,253.85,241.76,4.76,12.09,7.00
75%,77.94,8.00,22.45,471.35,448.90,4.76,22.45,8.50
max,99.96,10.00,49.65,1042.65,993.00,4.76,49.65,10.00


In [16]:
# check outliers.
fig = make_subplots(rows=len(df.select_dtypes(exclude=['object', 'datetime64[ns, UTC]']).columns),
                   cols=1,
                   shared_xaxes=False)

i = 1
for x in np.array(df.select_dtypes(exclude=['object', 'datetime64[ns, UTC]']).columns) :
    fig.add_trace(go.Box(x=df[x], name=x, boxpoints='suspectedoutliers'),
                  row=i,
                  col=1)
    i+=1

fig.show()

# Data Analysis

In [17]:
df.sample(2)

,invoice_id,branch,city,customer_type,gender,product_line,unit_price,quantity,tax_5%,total,payment,cogs,gross_margin_percentage,gross_income,rating,date_time
403,831-64-0259,B,Mandalay,Normal,Female,Fashion accessories,39.75,5,9.9375,208.6875,Ewallet,198.75,4.761905,9.9375,9.6,2019-02-22 10:43:00+00:00
532,857-67-9057,B,Mandalay,Normal,Male,Electronic accessories,22.95,10,11.4750,240.9750,Ewallet,229.50,4.761905,11.4750,8.2,2019-02-06 19:20:00+00:00


## General Insights  
- ***strong relation between groos income & cogs (cost of goods sold)***  
- ***strong relation between groos income & total***  
- ***strong relation between groos income & tax***  
- ***good relation between groos income & unit price***  
- ***good relation between groos income & quantity***

In [18]:
# visulization
px.imshow(df.corr(numeric_only=True),
          text_auto=True,
         aspect=True,
         color_continuous_scale='Blues',
         title='the correlation coefficient of numerical columns')

In [19]:
# visulization
px.scatter(data_frame=df,
           x='cogs',
           y='gross_income',
           size='tax_5%',
           color='payment',
           facet_col='payment',
           trendline='ols',
           title='relation between cost of goods sales & gross income & tax')

In [20]:
# visulization
px.scatter(data_frame=df,
           x='tax_5%',
           y='gross_income',
           size='total',
           color='product_line',
           facet_col='payment',
           trendline='ols',
           title='relation between cost of goods sales & gross income & tax')

## Insight 1  
- ***branch C makes the highest gross income which is 34.2 % of the total gross income***.

In [21]:
df.groupby('branch', as_index=False).agg({'gross_income': 'sum'})

,branch,gross_income
0,A,5057.1605
1,B,5057.0320
2,C,5265.1765


In [22]:
px.pie(data_frame=df.groupby('branch', as_index=False).agg({'gross_income': 'sum'}),
      names=df.groupby('branch', as_index=False).agg({'gross_income': 'sum'})['branch'],
      values=df.groupby('branch', as_index=False).agg({'gross_income': 'sum'})['gross_income'],
      title='percentage of groos income for each branch')

## Insight 2  
- ***member customer type makes the highest gross income which is 50.8 % of total gross income***

In [23]:
df.groupby('customer_type', as_index=False).agg({'gross_income': 'sum'})

,customer_type,gross_income
0,Member,7820.164
1,Normal,7559.205


In [24]:
px.pie(data_frame=df.groupby('customer_type', as_index=False).agg({'gross_income': 'sum'}),
      names=df.groupby('customer_type', as_index=False).agg({'gross_income': 'sum'})['customer_type'],
      values=df.groupby('customer_type', as_index=False).agg({'gross_income': 'sum'})['gross_income'],
      title='percentage of gross income for each customer type')

## Insight 3  
- ***the highest gross income comes from female customers which is 52 % of total gross income source.***

In [25]:
df.groupby('gender', as_index=False).agg({'gross_income': 'sum'})

,gender,gross_income
0,Female,7994.425
1,Male,7384.944


In [26]:
px.pie(data_frame=df.groupby('gender', as_index=False).agg({'gross_income': 'sum'}),
      names=df.groupby('gender', as_index=False).agg({'gross_income': 'sum'})['gender'],
      values=df.groupby('gender', as_index=False).agg({'gross_income': 'sum'})['gross_income'],
      title='percentage of gross income for male and female')

## Insight 4  
- ***Food and beverages product line makes the highest gross income with 17.4 % of the total gross income.***
- ***Sports and travel product line comes after with 17.1 %.***

In [27]:
df.groupby('product_line', as_index=False).agg({'gross_income': 'sum'}).sort_values(by='gross_income', ascending=False)

,product_line,gross_income
2,Food and beverages,2673.5640
5,Sports and travel,2624.8965
0,Electronic accessories,2587.5015
1,Fashion accessories,2585.9950
4,Home and lifestyle,2564.8530
3,Health and beauty,2342.5590


In [28]:
px.bar(df.groupby('product_line', as_index=False).agg({'gross_income': 'sum'}),
       x='product_line',
       y='gross_income',
       color='product_line',
      title='sum of gross income for each product line')

In [29]:
px.pie(data_frame=df.groupby('product_line', as_index=False).agg({'gross_income': 'sum'}),
      names='product_line',
      values='gross_income',
      title='percentage of sum of gross income for each product line')

## Insight 5  
- ***Cash payment method makes the highest gross income way with 34.7 % of the total gross income and Ewallet comes after with 34.1 %.***

In [30]:
df.groupby('payment', as_index=False).agg({'gross_income': 'sum'}).sort_values(by='gross_income', ascending=False)

,payment,gross_income
0,Cash,5343.170
2,Ewallet,5237.767
1,Credit card,4798.432


In [31]:
px.pie(data_frame=df.groupby('payment', as_index=False).agg({'gross_income': 'sum'}),
      names='payment',
      values='gross_income',
      title='percentage of sum of gross income for each payment method')

# Time Series Analysis

In [32]:
# reset index by date column.
df.set_index(df['date_time'], inplace=True)

In [33]:
# check changes.
df.sample(2)

,invoice_id,branch,city,customer_type,gender,product_line,unit_price,quantity,tax_5%,total,payment,cogs,gross_margin_percentage,gross_income,rating,date_time
date_time,,,,,,,,,,,,,,,,
2019-01-10 11:10:00+00:00,226-34-0034,B,Mandalay,Normal,Female,Electronic accessories,13.78,4,2.756,57.876,Ewallet,55.12,4.761905,2.756,9.0,2019-01-10 11:10:00+00:00
2019-03-20 19:17:00+00:00,142-63-6033,B,Mandalay,Normal,Male,Home and lifestyle,92.36,5,23.090,484.890,Ewallet,461.80,4.761905,23.090,4.9,2019-03-20 19:17:00+00:00


## Insight 1  
- ***all data records are in one year***  
- ***all records are in three months start from January and end with March.***

In [34]:
df.resample('Y').agg({'gross_income': 'sum'})

,gross_income
date_time,
2019-12-31 00:00:00+00:00,15379.369


In [35]:
df.resample('m').agg({'gross_income': 'sum'})

,gross_income
date_time,
2019-01-31 00:00:00+00:00,5537.708
2019-02-28 00:00:00+00:00,4629.494
2019-03-31 00:00:00+00:00,5212.167


## Insight 2  
- ***Saturday has the highest gross income and Thusday comes after.***  
- ***Monday has the lowest gross income.***

In [36]:
# resample df by day to calculate gross income for each day
df.resample('d').agg({'gross_income': 'sum'})

,gross_income
date_time,
2019-01-01 00:00:00+00:00,225.9610
2019-01-02 00:00:00+00:00,92.6430
2019-01-03 00:00:00+00:00,98.9585
2019-01-04 00:00:00+00:00,77.3185
2019-01-05 00:00:00+00:00,168.4135
...,...
2019-03-26 00:00:00+00:00,93.4530
2019-03-27 00:00:00+00:00,138.2295
2019-03-28 00:00:00+00:00,106.1620


In [37]:
# visulization
px.line(df.resample('d').agg({'gross_income': 'sum'}),
        markers=True,
        title='gross income by date')

In [38]:
# creat new column named 'day_name'
df['day_name'] = df.index.day_name()

In [39]:
# grouping df by 'day_name' column to calculate sum of gross income.
df.groupby('day_name', as_index=False).agg({'gross_income': 'sum'}).sort_values(by='gross_income', ascending=False)

,day_name,gross_income
2,Saturday,2672.4195
5,Tuesday,2451.5355
4,Thursday,2159.4880
3,Sunday,2117.0425
0,Friday,2091.7305
6,Wednesday,2082.4350
1,Monday,1804.7180


In [40]:
# visulization
px.bar(data_frame=df.groupby('day_name', as_index=False).agg({'gross_income': 'sum'}).sort_values(by='gross_income'),
      x='day_name',
      y='gross_income',
      color='day_name',
      title='sum of gross income by day of the week from January 2019 to March 2019')

## Insight 3  
- ***The shift from 12:00 AM to 8:00 AM has the highest gross income, and the shift from 8:00 AM to 16:00 PM comes after.***  
- ***In March the shift from 8:00 AM to 16:00 PM has the highest gross income most of times***.

In [41]:
# resample of df where month is January by period time equal 8 hr (for each shift) * 60 min for each hour to calculate gross income for each shift
df[df.index.month == 1].resample('480t').agg({'gross_income': 'sum'})

,gross_income
date_time,
2019-01-01 08:00:00+00:00,171.2020
2019-01-01 16:00:00+00:00,54.7590
2019-01-02 00:00:00+00:00,0.0000
2019-01-02 08:00:00+00:00,47.8775
2019-01-02 16:00:00+00:00,44.7655
...,...
2019-01-30 08:00:00+00:00,38.3975
2019-01-30 16:00:00+00:00,83.4245
2019-01-31 00:00:00+00:00,0.0000


In [42]:
# visulization
px.line(df[df.index.month == 1].resample('480T').agg({'gross_income': 'sum'}),
       markers=True,
       title='sum of gross income for each shift 8 hrs in month of January')

In [43]:
# resample of df where month is February by period time equal 8 hr (for each shift) * 60 min for each hour to calculate gross income for each shift
df[df.index.month == 2].resample('480t').agg({'gross_income': 'sum'})

,gross_income
date_time,
2019-02-01 08:00:00+00:00,38.2710
2019-02-01 16:00:00+00:00,78.1355
2019-02-02 00:00:00+00:00,0.0000
2019-02-02 08:00:00+00:00,85.2325
2019-02-02 16:00:00+00:00,111.9555
...,...
2019-02-27 08:00:00+00:00,121.3005
2019-02-27 16:00:00+00:00,157.7210
2019-02-28 00:00:00+00:00,0.0000


In [44]:
# visulization
px.line(df[df.index.month == 2].resample('480T').agg({'gross_income': 'sum'}),
       markers=True,
       title='sum of gross income for each shift 8 hrs in month of February')

In [45]:
# resample of df where month is January by period time equal 8 hr (for each shift) * 60 min for each hour to calculate gross income for each shift
df[df.index.month == 3].resample('480t').agg({'gross_income': 'sum'})

,gross_income
date_time,
2019-03-01 08:00:00+00:00,86.3645
2019-03-01 16:00:00+00:00,39.0815
2019-03-02 00:00:00+00:00,0.0000
2019-03-02 08:00:00+00:00,128.1640
2019-03-02 16:00:00+00:00,184.2315
...,...
2019-03-29 08:00:00+00:00,90.1450
2019-03-29 16:00:00+00:00,101.4380
2019-03-30 00:00:00+00:00,0.0000


In [46]:
# visulization
px.line(df[df.index.month == 3].resample('480T').agg({'gross_income': 'sum'}),
       markers=True,
       title='sum of gross income for each shift 8 hrs in month of March')